In [1]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
ce = CrackPropagation(n_seg=28)
ce.sz_bd.matrix='advanced'
ce.sz_cp.trait_set(x_00=ce.sz_bd.L * 0.3, n_m = 15)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=300, B=200, L=1000)
#ce.sz_bd.cross_section_layout.reinforcement[0].trait_set(z=20, A=1, E=1)

In [2]:
ce.run()
ce.interact()

seg 0
seg 1
seg 2
seg 3
seg 4
seg 5
seg 6
seg 7
seg 8
seg 9


<lambdifygenerated-21>:2: RuntimeWarning: invalid value encountered in sqrt
  return select([less(f_c, -35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w),less_equal(w, 0.028*d_a**0.32*f_c**0.18/f_t),less_equal(w, 0.14392*d_a**0.32*f_c**0.18/f_t),True], [-f_c,35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w,f_t*exp(-35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t*(-0.028*d_a**0.32*f_c**0.18/f_t + w)),select([greater(w, 0.028*d_a**0.32*f_c**0.18/f_t)], [-0.62*s*(s**2/w**2 + 1)**(-0.25)*select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan)/sqrt(w)], default=nan)], default=nan)
<lambdifygenerated-22>:2: RuntimeWarning: invalid value encountered in sqrt
  return select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan)
<lambdifygenerated-20>:2: RuntimeWarning: i

no convergence
stopped after 9


AttributeError: 'RCBeamDesign' object has no attribute 'z_N'

How to construct an animation?

- Each model with a progress bar records the history. Thus, the history browsing might be defines at an abstract level.

# Access the plot functions of the components

In [ ]:
import matplotlib.pylab as plt
fig, ax = plt.subplots(1,1,figsize=(10,4))
bd = ce.sz_cp.sz_bd
ax.set_ylim(ymin=0 ,ymax=bd.H)
ax.set_xlim(xmin=-0.05*bd.L ,xmax=1.05*bd.L)
ax.axis('equal');
bd.plot_sz_bd(ax)
ds = ce.crack_tip_orientation.crack_tip_shear_stress.sz_stress_profile.ds
ds.sz_ctr.plot_crack_tip_rotation(ax)
ce.sz_cp.plot_sz0(ax)
ds.plot_sz1(ax)
ds.plot_sz_fill(ax)

# Plot the crack path by looping over its nodes

In [ ]:
ce.sz_cp.x_t_Ia[0,:]

In [ ]:
from bmcs_shear.shear_crack.crack_extension import CrackExtension
cext = CrackExtension()
cext.sz_cp.x_00 = ce.sz_cp.x_t_Ia[0,0]
cext.sz_cp.x_t_Ia[0,:]

In [ ]:
%%capture
import matplotlib.pylab as plt
import numpy as np
from matplotlib import animation, rc
fig, (ax, ax_F) = plt.subplots(1,2,figsize=(8,3), tight_layout=True)
ds = ce.crack_tip_orientation.crack_tip_shear_stress.sz_stress_profile.ds
sz_cp = ce.sz_cp
x_n_a = sz_cp.x_t_Ia[0,:]
x_Ia = sz_cp.x_t_Ia
n_extend = 10
F_beam = np.array(ce.F_beam)
v_beam = np.array(ce.v_n)
v_beam = np.hstack([v_beam, v_beam[-1]*np.ones((n_extend,))])
F_beam = np.hstack([F_beam, F_beam[-1]*np.ones((n_extend,))])
x_Ia = np.vstack([x_Ia, x_Ia[-1,:]*np.ones((n_extend,2))])

ds.plot_sz_fill(ax)
ax_F.set_xlim(xmax=1.04*np.max(v_beam))
ax_F.set_ylim(ymax=1.04*np.max(F_beam))
v_n = v_beam[0]
F_n = F_beam[0]

def plot_crack_step(i):
    global x_n_a, v_n, F_n
    x_n1_a = x_Ia[i,:]
    ax.plot([x_n_a[0],x_n1_a[0]], [x_n_a[1], x_n1_a[1]], '-', color='black')
    ax.axis('equal');
    x_n_a = x_n1_a
    v_n1, F_n1 = v_beam[i], F_beam[i]
    ax_F.plot([v_n,v_n1], [F_n, F_n1], 'o-', color='blue')
    v_n, F_n = v_n1, F_n1

In [ ]:
def html5_crack_propagate_video():
    n_t = len(x_Ia)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_crack_step,
                                   frames=n_t, interval=200, blit=True, repeat=True, repeat_delay=2000)
    return anim.to_html5_video()

In [ ]:
import os.path
home_dir = os.path.expanduser('~')
anim_dir = os.path.join(home_dir, 'generated_animation')
if not os.path.exists(anim_dir):
    os.makedirs(anim_dir)

In [ ]:
%%capture
crack_propagate_file = os.path.join(anim_dir, 'crack_propagate_animation.html')
with open(crack_propagate_file,'w') as html_video_file:
    html_video_file.write(html5_crack_propagate_video())

In [ ]:
from IPython.display import HTML
html_video_file = open(crack_propagate_file,'r')
HTML(html_video_file.read())

# TODO
 - Check if the crack segments are picked up correctly
 - Check if the L_fps should be plotted as well at the end?